# 3D printing with Jubilee
Let's start by initiating communication with the machine and define the gel extruder(s).

In [1]:
%load_ext autoreload
%autoreload 2
from science_jubilee.Machine import Machine
from science_jubilee.tools.Tool import Tool
from science_jubilee.tools.SyringeExtruder import SyringeExtruder
from science_jubilee.tools.Camera import Camera

In [2]:
m = Machine(address = "jubilee.local")

/home/pi/POSE/science_jubilee/science_jubilee/Machine.py:332: UserWarning: Connection initiated with tool equipped. Use reload_tool() after instantiate this tool.
  warnings.warn("Connection initiated with tool equipped. Use reload_tool() after instantiate this tool.")


The first thing you do is to home the machine!

In [ ]:
m.home_all()

Homing could take a minute. Now, define and load tool(s).

In [ ]:
# Change your tool numbers to match your machine!
syringe_0 = SyringeExtruder(0, "white syringe")
syringe_1 = SyringeExtruder(1, "orange syringe")
m.reload_tool(syringe_0) # if you reinitiate a tool, use reload_tool instead of load_tool
m.load_tool(syringe_1)
m.tools

3D printing fine structure is very sensitive to the z offset. Z offset changes with the nozzle you're using, the length of the syringe, etc. Now we need to zero the nozzle tip and update the z offset.

First, pick up the tool you'd like to calibrate.

In [ ]:
m.pickup_tool(0)

The z offset on startup is an overshoot to prevent tool crashing. Let's move to the current `Z = 0`.

In [ ]:
m.move_to(z = 0)

Science Jubilee would prevent you from going further! Use the method `approach` inside `SyringeExtruder` to bring the tip of the syringe extruder into contact with the print bed. The z offset is automatically updated with `approach` as you do so.

In [ ]:
syringe_0.approach(-0.01) # you may need to approach many times to get the right position
print(m.tool_z_offsets[0])

If it says `"MachineStateError: Error: Relative move exceeds Z axis limit!"`, temporarily update the z offset to allow the nozzle to go lower. 

Before loading your slicer-generated gcode file, it's a good practice to prime the nozzle - extrude a little bit of material so that the nozzle is filled and ready to print.

Noted that the parameter of `extrude` is the length of the filament, i.e. the actual plunger movement. 

In [ ]:
m.move(dz = 25) # move the nozzle up if it's too close to the bed
syringe_0.extrude(0.1) # you may need to extrude many times. An empty tapered nozzle needs ~1.5 to fill up

Helper functions to load gcode and print gcode:

In [ ]:
# load the .gcode file and parse
def load_gcode(file_path):
    try:
        lines = []
        with open(file_path, 'r') as file:
            for line in file:
                lines.append(line.strip())
        return lines
    except FileNotFoundError:
        print(f"File '{file_path}' not found.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    
def print_gcode(gcode):
    for line in gcode:
        if len(line) > 0:
            if not line.startswith(';'):
                print(line)
                m.gcode(line)


In [ ]:
gcode = load_gcode("cylinder-20mm.gcode")
print_gcode(gcode)

In [ ]:
m.park_tool()

In [ ]:
## Testing images after each layer

In [4]:
camera = Camera(2, 'camera')
m.reload_tool(cam)

syringe = SyringeExtruder(1, 'extruder1', config="10cc_syringe")
m.load_tool(syringe)

In [ ]:
m.pickup_tool(camera)

In [ ]:
m.move_to(x=100, y=100) 

In [7]:
f = cam.get_frame()
cam.show_frame(f, grid=True)

In [ ]:
# try a nonplanar cube
z = 0.2
z_off = 0
start_x = 220
start_y = 220
side_length = 20
m.move_to(x=start_x, y=start_y, z=0.1)
for layer in range(10):
    syringe.move_extrude(x = start_x + side_length, y = start_y, z = z, multiplier = 2)
    syringe.move_extrude(x = start_x + side_length, y = start_y - side_length, z = z + z_off, multiplier = 3)
    syringe.move_extrude(x = start_x, y = start_y - side_length, z = z + z_off, multiplier = 2)
    syringe.move_extrude(x = start_x, y =start_y, z = z, multiplier = 3)
    z += 0.2
    z_off += 0.1

In [ ]:
## try wavy print
import math
import numpy as np
def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0:
        return v
    return v / norm

def make_wave(point, center, theta, phase):
    amplitude = 1
    frequency = 6
    
    # get unit vector in direction of current point from center
    point_arr = np.array(point)
    center_arr = np.array(center)
    direction = normalize(np.subtract(point_arr, center_arr))
    
    # push the point in this direction by an amt determined by current angle
    sine_off_mag = amplitude * math.sin(frequency * theta + phase)
    sine_off = np.multiply(sine_off_mag, direction)
    offset_point = np.add(point, sine_off)
    
    return offset_point

In [ ]:
%matplotlib inline
z = 0.1
theta = 0
center_x = 230
center_y = 210
center = [center_x, center_y]
radius = 10
theta_step = 2 * math.pi / 50
phase = 0

m.move_to(x=center_x, y=center_y)
while z < 20:
    m.pickup_tool(syringe)
    m.move_to(x=center_x, y=center_y)
    while theta < 2 * math.pi:
        point = [radius * math.cos(theta) + center_x, radius * math.sin(theta) + center_y]
        wave_point = make_wave(point, center, theta, phase)
        if theta==0:
            m.move_to(x = wave_point[0], y = wave_point[1], z = z)
        else:
            syringe.move_extrude(x = wave_point[0], y = wave_point[1], z = z, multiplier = 3)
        theta += theta_step
    z += 0.4
    phase += math.pi/2
    theta = 0
    
    # now take an image
    m.move_to(z=20)
    m.pickup_tool(cam)
    m.move_to(z=30+z) # focus height above top of print
    m.move_to(x=230, y=170) # need to do xy alignment
    frame = cam.get_frame()
    cam.show_frame(frame, save=True, save_path=f'/home/pi/syringe_z{z}.jpg')
    

In [ ]:
cam.video_stream()

In [ ]:
frame = cam.get_frame()
cam.show_frame(frame, save=True, save_path=f'/home/pi/syringe_z{z}.jpg')